## Knowledge distillation from CLIP

- [ ] Obtain data
- [ ] Transform eeg into spectrogram
- [ ] Extract CLIP features from images
- [ ] Train a classifier from CLIP features
- [ ] Build a CNN classifier from images
- [ ] Do knowledge distillation

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
import os 
from os.path import join as opj
from datasets import load_dataset
import tqdm

import pytorch_lightning as pl
from torch import nn
from torchsummary import summary
from pytorch_lightning import Trainer
from versatile_diffusion_dual_guided_fake_images import *
from transformers import AutoProcessor, AutoModelForCausalLM
from sklearn.linear_model import Ridge, RidgeCV
import torch.nn.functional as F
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from scipy.signal import stft
from sklearn.linear_model import RidgeClassifier
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report
from monai.networks import nets
import wandb
import pandas as pd
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
import copy
import seaborn as sns
# from pl_bolts.models.autoencoders.components import (
#     resnet18_decoder,
#     resnet18_encoder,
# )


In [ ]:
wandb.login()

In [ ]:
config={"method": "spectrogram"}

wandb.init(project="EEG_Decoding", config=config)

## 0. Obtain data

In [ ]:
dataset_dir="/home/matteo/data/EEG/ImageNetEEG"
imagenet_dir="/home/matteo/data/ImageNet/ILSVRC/Data/CLS-LOC/"

file=opj(dataset_dir,"eeg_14_70_std.pth")
data=torch.load(file)

block=torch.load(opj(dataset_dir,"block_splits_by_image_all.pth"))

In [ ]:
eeg_train=[]
eeg_val=[]
eeg_test=[]

subject_train=[]
subject_val=[]
subject_test=[]


img_train=[]
img_val=[]
img_test=[]

label_train=[]
label_val=[]
label_test=[]

start=40
stop=480

for i in range(11965):
    eeg=data["dataset"][i]["eeg"][:,start:stop]
    img_idx=data["dataset"][i]["image"]
    img_name=data["images"][img_idx]
    label=data["dataset"][i]["label"]
    subject=data["dataset"][i]["subject"]
    
    if i in block["splits"][0]["train"]:
        eeg_train.append(eeg)
        img_train.append(img_name)
        label_train.append(label)
        subject_train.append(subject)
        
    elif i in block["splits"][0]["val"]:
        eeg_val.append(eeg)
        img_val.append(img_name)
        label_val.append(label)
        subject_val.append(subject)
        
    elif i in block["splits"][0]["test"]:
        eeg_test.append(eeg)
        img_test.append(img_name)
        label_test.append(label)
        subject_test.append(subject)
        
    
eeg_train=torch.stack(eeg_train,0)
eeg_val=torch.stack(eeg_val,0)
eeg_test=torch.stack(eeg_test,0)

img_train = [opj(imagenet_dir,"train",i.split("_")[0],f"{i}.JPEG") for i in img_train]
img_val = [opj(imagenet_dir,"train",i.split("_")[0],f"{i}.JPEG") for i in img_val]
img_test = [opj(imagenet_dir,"train",i.split("_")[0],f"{i}.JPEG") for i in img_test]


## 1. Transform EEG into spectrogram images

In [ ]:
spectro_train=[]
spectro_val=[]
spectro_test=[]

for eeg in tqdm.tqdm(eeg_train,0):
    f, t, Zxx = stft(eeg, 1000, nperseg=40)
    spectro_train.append(torch.tensor(np.abs(Zxx)))
                         
for eeg in tqdm.tqdm(eeg_val,0):
    f, t, Zxx = stft(eeg, 1000, nperseg=40)
    spectro_val.append(torch.tensor(np.abs(Zxx)))
    
for eeg in tqdm.tqdm(eeg_test,0):
    f, t, Zxx = stft(eeg, 1000, nperseg=40)
    spectro_test.append(torch.tensor(np.abs(Zxx)))

In [ ]:
spectro_train=torch.stack(spectro_train)
spectro_val=torch.stack(spectro_val)
spectro_test=torch.stack(spectro_test)


## 2. Extract CLIP features from images 
Also extract ResNet50 represntation as backup

In [ ]:
device="cuda:0"

In [ ]:
pipe_embed = VersatileDiffusionDualGuidedFromCLIPEmbeddingPipeline.from_pretrained("shi-labs/versatile-diffusion", )

pipe_embed.remove_unused_weights()
pipe_embed = pipe_embed.to(device)

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)
# Set the model to evaluation mode
resnet50.eval()
resnet50=resnet50.to(device)

# Define the image transformations to be applied to each input image
transform_res = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

resnet_feature_extractor = create_feature_extractor(resnet50, return_nodes=["avgpool"])


In [ ]:
batch=64

In [ ]:
train_clip_img_embeds=[]
train_resnet=[]



with torch.no_grad():
    for i in tqdm.tqdm(range(0,len(eeg_train),batch)):

        #save img data
        y= img_train[i:i+batch]
                         
        images=[Image.open(i).convert("RGB") for i in y]

        #encode images in CLIP
        image_features=pipe_embed._encode_image_prompt(images,device=device,num_images_per_prompt=1,do_classifier_free_guidance=False).cpu()
        train_clip_img_embeds.append(image_features)

                
        tr_imgs=torch.stack([transform_res(i) for i in images]).to(device)
        
        out=resnet_feature_extractor(tr_imgs)
        train_resnet.append(out["avgpool"].cpu())
        
        
    train_clip_img_embeds = torch.cat(train_clip_img_embeds,axis=0)
    train_resnet = torch.cat(train_resnet,axis=0)

In [ ]:
val_clip_img_embeds=[]
val_resnet=[]



with torch.no_grad():
    for i in tqdm.tqdm(range(0,len(eeg_val),batch)):

        #save img data
        y= img_val[i:i+batch]
                         
        images=[Image.open(i).convert("RGB") for i in y]

        #encode images in CLIP
        image_features=pipe_embed._encode_image_prompt(images,device=device,num_images_per_prompt=1,do_classifier_free_guidance=False).cpu()
        val_clip_img_embeds.append(image_features)

                
        tr_imgs=torch.stack([transform_res(i) for i in images]).to(device)
        
        out=resnet_feature_extractor(tr_imgs)
        val_resnet.append(out["avgpool"].cpu())
        
        
    val_clip_img_embeds = torch.cat(val_clip_img_embeds,axis=0)
    val_resnet = torch.cat(val_resnet,axis=0)

In [ ]:
test_clip_img_embeds=[]
test_resnet=[]



with torch.no_grad():
    for i in tqdm.tqdm(range(0,len(eeg_test),batch)):

        #save img data
        y= img_test[i:i+batch]
                         
        images=[Image.open(i).convert("RGB") for i in y]

        #encode images in CLIP
        image_features=pipe_embed._encode_image_prompt(images,device=device,num_images_per_prompt=1,do_classifier_free_guidance=False).cpu()
        test_clip_img_embeds.append(image_features)

                
        tr_imgs=torch.stack([transform_res(i) for i in images]).to(device)
        
        out=resnet_feature_extractor(tr_imgs)
        test_resnet.append(out["avgpool"].cpu())
        
        
    test_clip_img_embeds = torch.cat(test_clip_img_embeds,axis=0)
    test_resnet = torch.cat(test_resnet,axis=0)

## 3. Train a CLIP Classifier

In [ ]:
train_clip_img_embeds.shape

In [ ]:
clf = RidgeClassifier().fit(train_clip_img_embeds[:,0,:].numpy(), label_train)

In [ ]:
clf.score(val_clip_img_embeds[:,0,:].numpy(),label_val)

In [ ]:

class LinearClassifier(pl.LightningModule):
    def __init__(self,n_classes, lr=3e-4):
        super(LinearClassifier, self).__init__()

        self.n_classes = n_classes
        self.lr = lr

        self.fc = nn.LazyLinear( n_classes)

        self.loss = nn.CrossEntropyLoss()


    def forward(self, x):
        logits = self.fc(x)
        return logits

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        logits = self.forward(inputs)
        loss = self.loss(logits, targets)

        preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
        acc = torch.sum(preds == targets).item() / targets.size(0)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        logits = self.forward(inputs)
        loss = self.loss(logits, targets)

        preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
        acc = torch.sum(preds == targets).item() / targets.size(0)

        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [ ]:
clip_head_classifier=LinearClassifier(n_classes=40)

In [ ]:
clip_features_train_dataset=torch.utils.data.TensorDataset(train_clip_img_embeds[:,0,:],torch.tensor(label_train))
clip_features_val_dataset=torch.utils.data.TensorDataset(val_clip_img_embeds[:,0,:],torch.tensor(label_val))
clip_features_test_dataset=torch.utils.data.TensorDataset(test_clip_img_embeds[:,0,:],torch.tensor(label_test))

# Create dataloaders
train_loader = DataLoader(clip_features_train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(clip_features_val_dataset, batch_size=batch)
test_loader = DataLoader(clip_features_test_dataset, batch_size=batch)


In [ ]:
trainer = Trainer(devices=1,max_epochs=5)
trainer.fit(clip_head_classifier, train_loader, val_loader)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_input_channels: int, base_channel_size: int, latent_dim: int, act_fn: object = nn.GELU):
        """
        Args:
           num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
           base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
           latent_dim : Dimensionality of latent representation z
           act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2),  # 32x32 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 16x16 => 8x8
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 8x8 => 4x4
            act_fn(),
            nn.Conv2d(2 * c_hid, 2 * c_hid, kernel_size=3, padding=1, stride=2),  # 8x8 => 4x4
            act_fn(),
            nn.Flatten(),  # Image grid to single feature vector
            nn.LazyLinear(latent_dim),
        )

    def forward(self, x):
        return self.net(x)
    


class CNNClassifier(pl.LightningModule):
    def __init__(self, cnn_model,teaching_clip_head=None, lr=1e-3,scale=1., distill_knowledge=True):
        super(CNNClassifier, self).__init__()

        self.cnn_model = cnn_model
        self.teaching_clip_head=teaching_clip_head
        self.lr = lr
        self.scale=scale
        self.softmax=torch.nn.Softmax(-1)
        self.distill_knowledge=distill_knowledge


        self.loss = nn.CrossEntropyLoss()


    def forward(self, x):
        return self.cnn_model(x)

    def training_step(self, batch, batch_idx):
        inputs, features, targets = batch
        outputs = self.forward(inputs)
        
        loss_base = self.loss(outputs,targets)

        if self.distill_knowledge:
            teacher_outputs=self.teaching_clip_head(features)

            outputs_prob=self.softmax(outputs)
            teacher_prob=self.softmax(teacher_outputs)

            loss_teacher = self.loss(outputs_prob, teacher_prob)*self.scale
            loss_base = self.loss(outputs,targets)

            loss = loss_base + loss_teacher
        else:
            loss = loss_base
        _, predicted = torch.max(outputs, 1)
        acc = torch.sum(predicted == targets).item() / targets.size(0)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        inputs, features, targets = batch
        outputs = self.forward(inputs)
        
        loss_base = self.loss(outputs,targets)

        if self.distill_knowledge:
            teacher_outputs=self.teaching_clip_head(features)

            outputs_prob=self.softmax(outputs)
            teacher_prob=self.softmax(teacher_outputs)

            loss_teacher = self.loss(outputs_prob, teacher_prob)*self.scale
            loss_base = self.loss(outputs,targets)

            loss = loss_base + loss_teacher
        else:
            loss = loss_base
        _, predicted = torch.max(outputs, 1)
        acc = torch.sum(predicted == targets).item() / targets.size(0)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr,weight_decay=1e-2)
        return optimizer

## 4. Train a CNN

In [ ]:
class KnowledgeClassifierDataset(torch.utils.data.Dataset):
    def __init__(self,spectro,features,labels,spec_transform=None):
        super().__init__()
        self.spectro=spectro
        self.features=features
        self.labels=labels
        self.spec_transform=spec_transform
        
    def __len__(self):
        return len(self.spectro)

    def __getitem__(self, idx):
        
        
        eeg=self.spectro[idx]
        feat=self.features[idx]
       
        if self.spec_transform:
            eeg=self.spec_transform(eeg)
        
        
        labels=self.labels[idx]
        
        return eeg,feat,labels

In [ ]:
train_clip_img_embeds.shape

## 5. Do Knowledge Distillation

In [ ]:
subj_train_array=np.array(subject_train)
subj_val_array=np.array(subject_val)
subj_test_array=np.array(subject_test)


In [ ]:
# spectro_train[subj_train_array==subj_idx]
# label_train[subj_train_array==subj_idx]

In [ ]:
spec_transforms=torchvision.transforms.Compose([torchvision.transforms.Resize((32,32))])
single_subject=True
subj_idx=1

for subj_idx in [1,2,3,4,5,6]:
    
    print(f"[INFO] Processing {subj_idx}")
    if not single_subject:

        train_know_dataset = KnowledgeClassifierDataset(spectro_train, train_clip_img_embeds[:,0,:],label_train,spec_transform=spec_transforms)
        val_know_dataset = KnowledgeClassifierDataset(spectro_val, val_clip_img_embeds[:,0,:],label_val,spec_transform=spec_transforms)
        test_know_dataset = KnowledgeClassifierDataset(spectro_test, test_clip_img_embeds[:,0,:],label_test,spec_transform=spec_transforms)
    else:
        train_know_dataset = KnowledgeClassifierDataset(spectro_train[subj_train_array==subj_idx], train_clip_img_embeds[:,0,:][subj_train_array==subj_idx],torch.tensor(label_train)[subj_train_array==subj_idx],spec_transform=spec_transforms)
        val_know_dataset = KnowledgeClassifierDataset(spectro_val[subj_val_array==subj_idx], val_clip_img_embeds[:,0,:][subj_val_array==subj_idx],torch.tensor(label_val)[subj_val_array==subj_idx],spec_transform=spec_transforms)
        test_know_dataset = KnowledgeClassifierDataset(spectro_test[subj_test_array==subj_idx], test_clip_img_embeds[:,0,:][subj_test_array==subj_idx],torch.tensor(label_test)[subj_test_array==subj_idx],spec_transform=spec_transforms)


    train_know_dataloader=torch.utils.data.DataLoader(train_know_dataset,batch_size=64,shuffle=True)
    val_know_dataloader=torch.utils.data.DataLoader(val_know_dataset,batch_size=64,shuffle=False)

    test_know_dataloader=torch.utils.data.DataLoader(test_know_dataset,batch_size=64,shuffle=False)

    ## model
    
    
    base_model = nets.Classifier(in_shape=(128,32,32),classes=40,channels=[64,64,128,128,128,128],strides=[2,1,2,1,2,2],kernel_size=3, num_res_units=1, act='GELU', norm="BATCH", dropout=0.)
    # base_model=EEGConvClfModel(channels=[128, 128,128])
    cnn= CNNClassifier(base_model,teaching_clip_head=clip_head_classifier,lr=3e-4,scale=1,distill_knowledge=True)
    summary(cnn, (128,32,32),device="cpu")
   
    early_stop_callback = EarlyStopping(
       monitor='val_acc',
       min_delta=0.00,
       patience=10,
       verbose=False,
       mode='max'
    )
    trainer = Trainer(devices=1,max_epochs=50,callbacks=[early_stop_callback])
    trainer.fit(cnn, train_know_dataloader, val_know_dataloader)
    trainer.save_checkpoint(f"models/subj_{subj_idx}.pt")
    
    y_pred=[]
    y_true=[]
    with torch.no_grad():
        for spectro, feat, labels in tqdm.tqdm(test_know_dataloader,position=0):
            y_pred.append(cnn(spectro))
            y_true.append(labels)

        y_pred=torch.cat(y_pred)
        y_true=torch.cat(y_true)

    report=classification_report(y_true.numpy(),y_pred.argmax(-1).numpy(),output_dict=True)
    df = pd.DataFrame(report).transpose()
    print(df)
    wandb.log({f"report_{subj_idx}": df})

In [ ]:
# cnn=Encoder(num_input_channels=128,base_channel_size=64,latent_dim=40)
# # cnn=CNNRegressor(cnn,lr=1e-3,scale=1e3)
# cnn= CNNClassifier(cnn,teaching_clip_head=clip_head_classifier,lr=3e-4,scale=1)
# summary(cnn, (128,32,32),device="cpu")

# base_model = nets.Classifier(in_shape=(128,32,32),classes=40,channels=[64,64,128,128,128,128],strides=[2,1,2,1,2,2],kernel_size=3, num_res_units=1, act='GELU', norm="BATCH", dropout=0.1)
# # base_model=EEGConvClfModel(channels=[128, 128,128])
# cnn= CNNClassifier(base_model,teaching_clip_head=clip_head_classifier,lr=3e-4,scale=1,distill_knowledge=True)
# summary(cnn, (128,32,32),device="cpu")

In [ ]:
# trainer = Trainer(devices=1,max_epochs=50)
# trainer.fit(cnn, train_know_dataloader, test_know_dataloader)

In [ ]:
# y_pred=[]
# y_true=[]
# for spectro, feat, labels in tqdm.tqdm(test_know_dataloader,position=0):
#     y_pred.append(cnn(spectro))
#     y_true.append(labels)

In [ ]:
# y_pred=torch.cat(y_pred)
# y_true=torch.cat(y_true)


In [ ]:
# subj_test_array=np.array(subject_test)


In [ ]:
# report=classification_report(y_true.numpy(),y_pred.argmax(-1).numpy(),output_dict=True)
# df = pd.DataFrame(report).transpose()
# df

# wandb.log({f"report_{subj_idx}": df})

In [ ]:
# df

### 5B Play a little bit and explore different networks

In [ ]:

# class EEGConvClfModel(pl.LightningModule):
#     def __init__(self, in_channels=128, channels=[128, 256, 384], output_dim=40, do=0.3,lr=1e-4):
#         super(EEGConvClfModel, self).__init__()

#         self.channels = channels
#         self.output_dim = output_dim
#         self.num_layers = len(channels)
#         self.lr = lr
#         self.validation_step_outputs = []
#         self.train_step_outputs = []

#         conv_layers = []
#         seq_length = 768

#         for i in range(self.num_layers):
#             out_channels = channels[i]
#             conv_layers.extend([
#                 nn.Conv1d(in_channels, out_channels, kernel_size=12, stride=4, padding=1),
#                 nn.GELU(),
#                 nn.BatchNorm1d(out_channels)
#             ])
#             in_channels = out_channels
#             seq_length //= 2

#         self.conv_layers = nn.Sequential(*conv_layers,nn.Dropout(do))
        
        
        
#         self.fc_layers = nn.LazyLinear(output_dim)

#         self.loss = nn.CrossEntropyLoss()

#         self.train_epoch_loss = []  # List to store training loss values per epoch
#         self.val_epoch_loss = []    # List to store validation loss values per epoch

#     def forward(self, x):
#         x = self.conv_layers(x)
#         x = torch.flatten(x, start_dim=1)
#         x = self.fc_layers(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         inputs, targets = batch
#         outputs = self.forward(inputs)
#         loss = self.loss(outputs, targets)
        
#         _, predicted = torch.max(outputs, 1)

#         acc = torch.sum(predicted == targets).item() / targets.size(0)
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

#         # self.train_step_outputs.append(loss)

#         return loss

#     def validation_step(self, batch, batch_idx):
#         inputs, targets = batch
#         outputs = self.forward(inputs)
#         loss = self.loss(outputs, targets)
        
#         _, predicted = torch.max(outputs, 1)

#         acc = torch.sum(predicted == targets).item() / targets.size(0)
#         self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
#         self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

#         # self.validation_step_outputs.append(loss)

#         return loss


#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(self.parameters(), self.lr, weight_decay=1e-2)
#         lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5)
#         return {
#             "optimizer": optimizer,
#             "lr_scheduler": lr_scheduler,
#             "monitor": "val_loss"
#         }


In [ ]:
# spec_transforms=torchvision.transforms.Compose([torchvision.transforms.Resize((32,32))])
# # spec_transforms=None
# single_subject=True
# subj_idx=1

# if not single_subject:
    
#     train_know_dataset = KnowledgeClassifierDataset(spectro_train, train_clip_img_embeds[:,0,:],label_train,spec_transform=spec_transforms)
#     test_know_dataset = KnowledgeClassifierDataset(spectro_test, test_clip_img_embeds[:,0,:],label_test,spec_transform=spec_transforms)
# else:
#     train_know_dataset = KnowledgeClassifierDataset(spectro_train[subj_train_array==subj_idx], train_clip_img_embeds[:,0,:][subj_train_array==subj_idx],torch.tensor(label_train)[subj_train_array==subj_idx],spec_transform=spec_transforms)
#     test_know_dataset = KnowledgeClassifierDataset(spectro_test[subj_test_array==subj_idx], test_clip_img_embeds[:,0,:][subj_test_array==subj_idx],torch.tensor(label_test)[subj_test_array==subj_idx],spec_transform=spec_transforms)


# train_know_dataloader=torch.utils.data.DataLoader(train_know_dataset,batch_size=64,shuffle=True)
# test_know_dataloader=torch.utils.data.DataLoader(test_know_dataset,batch_size=64,shuffle=False)


In [ ]:
# s,f,c=next(iter(train_know_dataloader))
# print(*s.shape[1:])

In [ ]:
# base_model = nets.Classifier(in_shape=(128,32,32),classes=40,channels=[64,64,128,128],strides=[2,2,2,2],kernel_size=5, num_res_units=1, act='GELU', norm='BATCH', dropout=0.3)
# # base_model=EEGConvClfModel(channels=[128, 128,128])
# cnn= CNNClassifier(base_model,teaching_clip_head=clip_head_classifier,lr=3e-4,scale=1,distill_knowledge=True)
# summary(cnn, (128,32,32),device="cpu")

In [ ]:
# trainer = Trainer(devices=1,max_epochs=20)
# trainer.fit(cnn, train_know_dataloader, test_know_dataloader)

### 5c. Masked Autoencoder

## 6. Compute metrics

In [ ]:
import nltk
from nltk.corpus import wordnet as wn

def convert_synsets_to_names(synsets):
    nltk.download('wordnet')  # Download WordNet corpus if not already downloaded

    class_names = []
    for synset_id in synsets:
        synset = wn.synset_from_pos_and_offset(synset_id[0], int(synset_id[1:]))
        class_name = synset.lemmas()[0].name()
        class_names.append(class_name)

    return class_names

In [ ]:
class_names=convert_synsets_to_names(data["labels"])

In [ ]:
syn_to_class = {k:v for k,v in zip(range(40),class_names)}

In [ ]:
syn_to_class

In [ ]:
idx=15
plt.imshow(Image.open(img_test[idx]))
plt.title(syn_to_class[label_test[idx]])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(preds, true_labels):
    # Compute accuracy
    accuracy = accuracy_score(true_labels, np.argmax(preds, axis=1))

    # Get the indices of the top 5 predicted classes for each sample
    top3_indices = np.argsort(preds, axis=1)[:, -3:]

    # Check if the true label is present in the top 5 predicted classes
    top3_accuracy = np.mean(np.any(top3_indices == true_labels.reshape(-1, 1), axis=1))


    # Get the indices of the top 5 predicted classes for each sample
    top5_indices = np.argsort(preds, axis=1)[:, -5:]

    # Check if the true label is present in the top 5 predicted classes
    top5_accuracy = np.mean(np.any(top5_indices == true_labels.reshape(-1, 1), axis=1))

    # Compute F1 score
    f1 = f1_score(true_labels, np.argmax(preds, axis=1), average='macro')

    # Compute precision
    precision = precision_score(true_labels, np.argmax(preds, axis=1), average='macro')

    # Compute recall
    recall = recall_score(true_labels, np.argmax(preds, axis=1), average='macro')

    return accuracy, top3_accuracy,top5_accuracy, f1, precision, recall


In [ ]:
subj_idx=1
metrics={}
for subj_idx in range(1,7):
    
    print(f"processing {subj_idx}")
    base_model = nets.Classifier(in_shape=(128,32,32),classes=40,channels=[64,64,128,128,128,128],strides=[2,1,2,1,2,2],kernel_size=3, num_res_units=1, act='GELU', norm="BATCH", dropout=0.)
    model= CNNClassifier(base_model,teaching_clip_head=clip_head_classifier,lr=3e-4,scale=1,distill_knowledge=True)
    model.load_state_dict(torch.load(f"models/subj_{subj_idx}.pt")["state_dict"])
    
    inputs= torch.stack([spec_transforms(i) for i in spectro_test[subj_test_array==subj_idx]])
    with torch.no_grad():
        preds=model(inputs)
    
    metric_values=compute_metrics(preds.numpy(),np.array(label_test)[subj_test_array==subj_idx])
    metrics[f"subj_{subj_idx}"]={"accuracy":metric_values[0],
                                 "top-3_accuracy":metric_values[1],
                                 "top-5_accuracy":metric_values[2],
                                 "f1":metric_values[3],
                                 "precision":metric_values[4],
                                 "recall":metric_values[5],
                                 "preds":preds.argmax(-1).numpy(),
                                 "labels":np.array(label_test)[subj_test_array==subj_idx]
                                }
                                 
                                 
                                 

In [ ]:
df=pd.DataFrame.from_dict(metrics)
df

In [ ]:
df_metrics=df.iloc[:5].transpose()

In [ ]:
df_metrics.to_csv("report.csv")

In [ ]:

# Set a theme
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
# Create a figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Create a barplot

sns.barplot( data=df_metrics.sort_values(by="subj_4",axis=1, ascending=False), palette='viridis', ax=ax,alpha=0.9)

# Add labels and title
ax.set_xlabel('Metric', fontsize=14)
ax.set_ylabel('Value', fontsize=14)
ax.set_title('CLIP-Guided EEG Decoding Performances', fontsize=18)

# Show the plot
plt.show()
plt.savefig("figures/performances.png")

In [ ]:
report=classification_report(np.array(label_test)[subj_test_array==subj_idx],preds.argmax(-1).numpy(),output_dict=True,target_names=class_names)
df = pd.DataFrame(report).transpose()
df


## 7. Prediction to images - just separate boxes

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)


In [ ]:
img_test_subj=np.array(img_test)[subj_test_array==subj_idx]

In [ ]:
metrics["subj_1"]

In [ ]:
decoded_images={}
test_images={}
for subj_idx in range(1,7):
    print(f"processing {subj_idx}")
    img_test_subj=np.array(img_test)[subj_test_array==subj_idx]
    key=f"subj_{subj_idx}"
    
    subj_imgs=[]
    for p in metrics[key]["preds"]:
        prompt = f"High quality photo of {syn_to_class[p]}"
        image = pipe(prompt, num_inference_steps=40,guidance_scale=9,negative_prompt="bad quality,poor resolution").images[0]
        subj_imgs.append(image)
    decoded_images[key]=subj_imgs
    test_images[key]=[Image.open(i) for i in img_test_subj]

In [ ]:
np.array(decoded_images[f"subj_1"]).__len__()

In [ ]:
offset=0
subj_idx=6

# selected_class=10

for selected_class in tqdm.tqdm(range(40),position=0):
    
    indices=np.array([metrics[f"subj_{subj_idx}"]["labels"]==selected_class]).squeeze()
    
    if indices.sum()>5:
        nrow=5
    else:
        nrow= indices.sum()
    fig, axs = plt.subplots(nrow,6,figsize=(20,20))

    for i in range(nrow):
        axs[i,0].imshow(np.array(test_images[f"subj_{subj_idx}"])[indices][i+offset])
        axs[i,0].axis("off")
        for j in range(2,7):
            axs[i,j-1].imshow(np.array(decoded_images[f"subj_{j}"])[indices][i+offset])
            axs[i,j-1].axis("off")
    axs[0,0].set_title("stimulus")
    for j in range(2,7):
        axs[0,j-1].set_title(f"subj_{j}")

    fig.tight_layout()
    fig.savefig(f"figures/{class_names[selected_class]}.png")

In [ ]:
indices.sum()

In [ ]:
np.array([metrics[f"subj_{subj_idx}"]["labels"]==selected_class])


In [ ]:
np.array(test_images[f"subj_{subj_idx}"])[np.array([metrics[f"subj_{subj_idx}"]["labels"]==selected_class]).squeeze()]

In [ ]:
fig, axs=plt.subplots(1,2,figsize=(20,10))

axs[0].imshow(Image.open(img_test_subj[idx]))
axs[1].imshow(image)

axs[0].axis("off")
axs[1].axis("off")


## 6B Predict on averages

In [ ]:
out=pipe.vae.decode(z).sample

In [ ]:
z.shape


In [ ]:
# out=top(out)

## 7. Extract latent from images

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Create the normalization transform
normalize_transform = torchvision.transforms.Normalize(mean, std)

tot=torchvision.transforms.Compose([torchvision.transforms.Resize((64,64)),torchvision.transforms.ToTensor()])
top=torchvision.transforms.ToPILImage()


In [ ]:
latents_train=[]
with torch.no_grad():
    for im in tqdm.tqdm(img_train,position=0):
        im=tot(Image.open(im).convert("RGB"))
        z=pipe.vae.encode(torch.tensor(im).unsqueeze(0).half().cuda()).latent_dist.mode()
        latents_train.append(z.cpu())
    latents_train= torch.cat(latents_train).flatten(start_dim=1)  

In [ ]:
latents_val=[]
with torch.no_grad():
    for im in tqdm.tqdm(img_val,position=0):
        im=tot(Image.open(im).convert("RGB"))
        z=pipe.vae.encode(torch.tensor(im).unsqueeze(0).half().cuda()).latent_dist.mode()
        latents_val.append(z.cpu())
    latents_val= torch.cat(latents_val).flatten(start_dim=1)

In [ ]:
latents_test=[]
with torch.no_grad():
    for im in tqdm.tqdm(img_test,position=0):
        im=tot(Image.open(im).convert("RGB"))
        z=pipe.vae.encode(torch.tensor(im).unsqueeze(0).half().cuda()).latent_dist.mode()
        latents_test.append(z.cpu())
    latents_test= torch.cat(latents_test).flatten(start_dim=1)

In [ ]:
feature_extractor=copy.deepcopy(model.cnn_model)
feature_extractor.final=nn.Flatten()
feature_extractor.reshape=nn.Identity()

In [ ]:
spectro_resized_subj_train= torch.stack([spec_transforms(i) for i in spectro_train[subj_train_array==subj_idx]])
with torch.no_grad():
    eeg_features_train=feature_extractor(spectro_resized_subj_train)

In [ ]:
spectro_resized_subj_test= torch.stack([spec_transforms(i) for i in spectro_test[subj_test_array==subj_idx]])
with torch.no_grad():
    eeg_features_test=feature_extractor(spectro_resized_subj_test)

In [ ]:
# regressor=RidgeCV(alphas=[1,10,100,1000,1e4,1e5],fit_intercept=True)
regressor=Ridge(alpha=0.1,fit_intercept=True)

regressor.fit(eeg_features_train.numpy(),latents_train[subj_train_array==subj_idx].numpy())

In [ ]:
preds=torch.tensor(regressor.predict(eeg_features_train.numpy())).reshape(-1,4,8,8)

In [ ]:
with torch.no_grad():
    a=pipe.vae.decode(preds[:10].cuda().half()).sample.cpu().float()
    b=pipe.vae.decode(latents_train[subj_train_array==subj_idx][:10].reshape(-1,4,8,8).cuda().half()).sample.cpu().float()
    

In [ ]:
fig,axs=plt.subplots(3,2)

for i in range(3):
    axs[i,0].imshow(top(a[i]))
    axs[i,1].imshow(top(b[i]))
    

## 9. Figures

## (Optional) Masked autoencoder for EEG data, neural latents

##  (Optional) Cebra for dimensionality reduction with CLIP latents

## 8. Try to regress out latents from EEG 

In [ ]:
# feature_extractor

In [ ]:
eeg_features_test.shape

In [ ]:
regressor=RidgeCV(alphas=[1,10,100,1000,1e4],fit_intercept=False)
regressor.fit(train_clip_img_embeds[:,0,:][subj_train_array==subj_idx].numpy(),eeg_features_train.numpy(),)

In [ ]:
inverse_regressor=np.linalg.pinv(regressor.coef_)

In [ ]:
estimated_clip_from_eeg_test_subj=eeg_features_test.numpy()@inverse_regressor.T

In [ ]:
with torch.no_grad():
    preds_from_estimated=clip_head_classifier(torch.tensor(estimated_clip_from_eeg_test_subj).float())

In [ ]:
report=classification_report(np.array(label_test)[subj_test_array==subj_idx],preds_from_estimated.argmax(-1).numpy(),output_dict=True,target_names=class_names)
df = pd.DataFrame(report).transpose()
df